In [12]:
import tensorflow as tf
import os,glob
import random

In [13]:
dataset_path = "/home/data/kbh/DNS-Challenge-16kHz/datasets_fullband/clean_fullband"
list_clean = glob.glob(os.path.join(dataset_path,"**",'*.wav'))
list_noise = glob.glob(os.path.join("/home/data/kbh/DNS-Challenge-16kHz/datasets_fullband/noise_fullband","*.wav"))
print(len(file_list))

223241


In [17]:
def mix(clean_path, noisy_path):
    clean_audio, _ = tf.audio.decode_wav(tf.io.read_file(clean_path))
    noisy_audio, _ = tf.audio.decode_wav(tf.io.read_file(noisy_path))
    clean_audio = tf.squeeze(clean_audio, axis=-1)  
    noisy_audio = tf.squeeze(noisy_audio, axis=-1) 
    if len(clean_audio) > 48000 :
        clean_audio = clean_audio[:48000]
    if len(noisy_audio) > 48000 :
        noisy_audio = noisy_audio[:48000]
    return clean_audio, noisy_audio

# 5. 데이터셋 생성 및 랜덤하게 샘플링하는 제너레이터 함수 정의
def data_generator():
    for _ in range(1000):  
        clean_path = random.choice(list_clean)
        noisy_path = random.choice(list_noise)  
        yield mix(clean_path, noisy_path)

# 6. Dataset을 생성하여 clean과 noisy 쌍을 포함
dataset = tf.data.Dataset.from_generator(
    data_generator,
    output_signature=(
        tf.TensorSpec(shape=(None,), dtype=tf.float32),  # 가변 길이 clean 오디오
        tf.TensorSpec(shape=(None,), dtype=tf.float32)   # 가변 길이 noisy 오디오
    )
)

# https://www.tensorflow.org/api_docs/python/tf/data/Dataset#padded_batch
# 필요시 max_length 설정, 데이터기 이것보다 길면 에러 발생 
# Attempted to pad to a smaller size than the input element
dataset = dataset.padded_batch(32, padded_shapes=([48000], [48000]))  

"""
https://www.tensorflow.org/guide/data_performance?hl=ko
특히 변환에 백그라운드 스레드와 내부 버퍼를 사용하여 요청된 시간보다 앞서 입력 데이터세트에서 요소를 미리 가져옵니다.
미리 가져올 요소의 수는 단일 학습 단계에서 사용되는 배치 수와 같거나 더 많아야 합니다. 
이 값을 수동으로 조정하거나, tf.data 런타임이 런타임에 값을 동적으로 조정하도록 tf.data.AUTOTUNE로 설정할 수 있습니다.
"""
dataset = dataset.prefetch(tf.data.AUTOTUNE)

# 8. 데이터셋 샘플 확인
for clean_batch, noisy_batch in dataset.take(1):
    print("Clean batch shape:", clean_batch.shape)
    print("Noisy batch shape:", noisy_batch.shape)

Clean batch shape: (32, 48000)
Noisy batch shape: (32, 48000)
